In [1]:
# This cell imports all necessary libraries for this code
import xarray as xr # library for loading netcdf files
import matplotlib.pyplot as plt # library for plotting 
import numpy as np # libary for working with arrays
import cartopy.crs as ccrs # library for plotting on maps
import matplotlib.ticker as mticker # library for putting axes labels

In [2]:
# This cell defines which file to read

# data that you want to load 
filename = 'era5.22N-32N.96W-83W.2005082900.nc'
# The first thing you want to do is to check what variables are stored in what formats. There are multiple ways to do so.
# method 1) go to terminal and type "ncdump -h filename" where the filename is the actual name of the file that you want to check.
# This will output information about store variables and attributes
# method 2) Check the file here using "xr.open_dataset(filename)".
# When you run this cell, the line below will output information about the data file
datafile = xr.open_dataset(filename)
datafile

ValueError: found the following matches with the input file in xarray's IO backends: ['netcdf4', 'h5netcdf']. But their dependencies may not be installed, see:
http://xarray.pydata.org/en/stable/user-guide/io.html 
http://xarray.pydata.org/en/stable/getting-started-guide/installing.html

In [ ]:
# This cell loads dimensions of the data
lat = datafile.lat.data # latitude (degress north)
lon = datafile.lon.data # longitude (degrees east)
lev = datafile.lev.data # pressure level (hPa)
time = datafile.time.data # time
print(time)

In [ ]:
# This cell will load variables of the data

# An example of loading one isobaric level (e.g., 850 hPa)
p0 = 850
u=datafile.u.sel(lev=p0).data[0,:,:] # zonal wind (m/s) with dimension lat x lon
v=datafile.v.sel(lev=p0).data[0,:,:] # meridional wind (m/s) with dimension lat x lon
g=9.8*datafile.g.sel(lev=p0).data[0,:,:] # geopotential (m^2 s^-2 since multiplied by gravity)
# for all these variables, I am only loading the first time index

# An example of loading surface variables (these variables do not have vertical dimension)
usfc = datafile.usfc.data[0,:,:] # 10-m zonal wind (m/s) with dimension lat x lon
vsfc = datafile.vsfc.data[0,:,:] # 10-m meridional wind (m/s)
mslp = datafile.mslp.data[0,:,:] # sea level pressure (Pa)


In [ ]:
# Plot a map with sea level pressure in filled contours and horizontal wind vectors
fig = plt.figure(figsize=(15,15)) # create a figure
plt.rcParams.update({'font.size':16}) # change the font size

projection = ccrs.PlateCarree() # specify axes and projection option (see Cartopy website for the full list of projection types)
ax = plt.axes(projection=projection) 

# plot filled contours
longrid,latgrid=np.meshgrid(lon,lat) # create matrices of lat and lon that match the size of the variable you want to plot
plot1 = ax.contourf(longrid,latgrid,mslp/100,transform=projection) # contour SLP
cbar = fig.colorbar(plot1,shrink=0.5) # add a colorbar
cbar.set_label('hPa') # add units label to the colorbar

# add vectors
vplot_int = 4 # grid interval to plot wind vectors (e.g., 1 for every grid, 2 for every other grid)
vscale = 50*np.mean((usfc[:]**2+vsfc[:]**2)**0.5) # scale parameter for the length of vectors
plot2 = ax.quiver(longrid[::vplot_int,::vplot_int],latgrid[::vplot_int,::vplot_int],\
                  usfc[::vplot_int,::vplot_int],vsfc[::vplot_int,::vplot_int],\
                  scale=vscale,transform=projection)
# add title
time_txt=str(time[0]) # converting time variable to a string
idx_date = time_txt.find('T') # finding 'T' in the time string (e.g. in 2006-12-14T00:00:00)
ax.set_title('Sea Level Pressure and Surface Winds: '+time_txt[0:idx_date]+' '+time_txt[idx_date+1:idx_date+6]) 

# add grid ticks
lontick = np.arange(-160,-135,5) # define longitude ticks
lattick = np.arange(35,50,5) # define latitude ticks
grl=ax.gridlines(crs=projection,draw_labels=True,color='k',alpha=0.5)
grl.top_labels = False
grl.right_labels = False
grl.xlocator = mticker.FixedLocator(lontick)
grl.ylocator = mticker.FixedLocator(lattick)

# to add x and y labels, add texts (set_xlabel and set_ylabel do not seem to work when you use Cartopy)
ax.text(np.mean(lon),np.min(lat)-1,'Longitude',ha='center',va='top')
ax.text(np.min(lon),np.mean(lat),'Latitude',ha='right',va='center',rotation=90)

# add streamlines to this figure
# You can use Python's 'streamplot' function or code your own